In [ ]:
###########################################################################
## Basic stuff
###########################################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))


###########################################################################
## Music
###########################################################################
from myMusicPathData import myMusicPathData


###########################################################################
## Utils
###########################################################################
from timeUtils import timestat
from fileIO import fileIO
from fsUtils import fsPath, fsInfo, dirUtil, fileUtil
from pandas import DataFrame, Series


###########################################################################
## DB
###########################################################################
from masterManualEntries import masterManualEntries
from masterArtistNameDB import masterArtistNameDB
from mainDB import mainDB
from matchArtistToDB import masterMatch

# General

In [ ]:
manDB      = masterArtistNameDB("main")
multimanDB = masterArtistNameDB("multi")
io   = fileIO()
mmpd = myMusicPathData()
mme  = masterManualEntries()
#maindb.setMasterDBData()   ### Full DB Access

# Find My Music

In [ ]:
mmpdData = mmpd.findMyMusic()

In [ ]:
mmpd.saveData(artistAlbums=mmpdData, local=False)

In [ ]:
#mmpdData = mmpd.getData()
mmpd.getSummary(mmpdData).head()

# Match To Global Manual Music DB

In [ ]:
mme      = masterManualEntries()
mmeDF    = mme.getData()
mmpdData = Series(mmpd.getData())
mm       = masterMatch()

In [ ]:
def nearMatches(x, cutoff=0.9):
    artistNameUpper = x.upper()
    searchResults = masterArtistNames.apply(mm.getLevenshtein, x2=artistNameUpper)
    return searchResults[searchResults > cutoff]

mmpdData           = Series(mmpd.getData())
masterArtistNames  = mmeDF["ArtistName"]
myMusicArtistNames = Series(mmpdData.index)

In [ ]:
ts = timestat("Finding Near Matches For {0} Unmatched Master Artists Using {1} Unknown List Artists".format(len(myMusicArtistNames), len(masterArtistNames)))
N = len(myMusicArtistNames)
matchResults = {}
for i,(idx,artistName) in enumerate(myMusicArtistNames.iteritems()):
    matchResults[artistName] = nearMatches(artistName)
    if (i+1) % 250 == 0 or (i+1) == 50:
        ts.update(n=i+1, N=N)
        break
#myMusicArtistNameMatches = myMusicArtistNames.apply(nearMatches)
ts.stop()
#print("Found {0} Possibly Matched Artists".format(aDFNear.sum()))

In [ ]:
results = masterArtistNames.apply(mm.getLevenshtein, x2="A Flock Of Seagulls")

In [ ]:
### Perfect Match
perfectMatches = {artistName: mmeDF[mmeDF["ArtistName"] == artistName] for idx,artistName in myMusicArtistNames.iteritems()}

In [ ]:
myMusicMatchIDs  = {}
noMusicMatchs    = {}
multiMusicMatchs = {}
for artistName,artistMatches in perfectMatches.items():
    if artistMatches.shape[0] == 1:
        myMusicMatchIDs[artistName] = list(artistMatches.index)[0]
    elif artistMatches.shape[0] > 1:
        print("Multiple Names: {0}".format(artistName))
        multiMusicMatchs[artistName] = artistMatches
    else:
        print("No Matches: {0}".format(artistName))
        noMusicMatchs[artistName] = True
        
myMusicMatchIDs  = Series(myMusicMatchIDs)
noMusicMatchs    = Series(noMusicMatchs)
multiMusicMatchs = Series(multiMusicMatchs)

In [ ]:
from masterDBData import masterDBData
mdbData = masterDBData(dtype="Search")
mdbData.loadArtists()
mdbData.loadAlbums()

In [ ]:
from listUtils import getFlatList

def getMasterAlbums(row):
    def getArtistDBAlbums(db,dbID):
        albums = mdbData.getArtistDBAlbumsFromID(db,dbID)
        retval = albums if albums is not None else []
        return retval    
    
    fullAlbumsList = getFlatList([getArtistDBAlbums(db,dbID) for db,dbID in row[row.notna()].iteritems()])
    retval = list(Series(fullAlbumsList).unique()) if len(fullAlbumsList) > 0 else []
    return retval

ts = timestat("Getting Master Artist Albums For {0} Entries".format(mmeDF.shape[0]))
masterArtistAlbums = mmeDF[mdbData.dbs].apply(getMasterAlbums, axis=1)
ts.stop()

In [ ]:
mmeDF["Albums"] = masterArtistAlbums

In [ ]:
masterArtistAlbumsData = mmeDF[["ArtistName", "Albums"]]

In [ ]:
masterArtistAlbumsData["NumAlbums"] = masterArtistAlbumsData["Albums"].apply(len)

In [ ]:
io.save(idata=masterArtistAlbums, ifile="masterArtistAlbums.p")

## Match My Music

In [ ]:
%autoreload
from matchRunParams import matchRunParams
from masterMatchGate import masterMatchGate
from matchDBData import matchDBData
from masterDBData import masterDBData

In [ ]:
dbToUse = "MyMusic"

io = fileIO()
artistSummaryDF     = io.get("music_artistSummaryDF.p")
myMusicArtistAlbums = DataFrame({artistName: {"ArtistName": artistName, "Albums": df["AlbumName"].to_list()} for artistName,df in artistSummaryDF.groupby("ArtistName")}).T
myMusicArtistAlbums["NumAlbums"] = myMusicArtistAlbums["Albums"].apply(len)

mdbDataDBToUse    = masterDBData(dtype="User", dbs="MyMusic")
mdbDataDBToUse.setUserDBData("MyMusic", myMusicArtistAlbums["ArtistName"], myMusicArtistAlbums["NumAlbums"], myMusicArtistAlbums["Albums"])
artistDBToUseData = mdbDataDBToUse.getDBBasicInfo(dbToUse)

In [ ]:
mme   = masterManualEntries()
mmeDF = mme.getData()
masterArtistAlbumsData              = mmeDF[["ArtistName"]]
masterArtistAlbumsData["Albums"]    = io.get("masterArtistAlbums.p")
masterArtistAlbumsData["NumAlbums"] = masterArtistAlbumsData["Albums"].apply(len)

In [ ]:


matchData = matchDBData(mmeDF=None, dtype=dtype, dbs=dbsToMatch)
matchData.loadArtists(Discogs=numAlbumsReq.get("Discogs",4),         MusicBrainz=numAlbumsReq.get("MusicBrainz",2), 
                      AllMusic=numAlbumsReq.get("AllMusic",3),       RateYourMusic=numAlbumsReq.get("RateYourMusic",2), 
                      Genius=numAlbumsReq.get("Genius",2),           AlbumOfTheYear=numAlbumsReq.get("AlbumOfTheYear",2), 
                      KWorbiTunes=numAlbumsReq.get("KWorbiTunes",2), KWorbSpotify=numAlbumsReq.get("KWorbiTunes",2))


matchData = masterDBData(dtype="User", dbs="Master")
matchData.setUserDBData("Master", masterArtistAlbumsData["ArtistName"], masterArtistAlbumsData["NumAlbums"], masterArtistAlbumsData["Albums"])

In [ ]:
matchData = masterDBData(dtype="User", dbs="Master")
matchData.setUserDBData("Master", masterArtistAlbumsData["ArtistName"], masterArtistAlbumsData["NumAlbums"], masterArtistAlbumsData["Albums"])

In [ ]:

artistDBToUseData = mdbDataDBToUse.getDBBasicInfo(dbToUse)

In [ ]:
%autoreload
from matchRunParams import matchRunParams
from masterMatchGate import masterMatchGate
from matchDBData import matchDBData
from masterDBData import masterDBData
from matchArtistToDB import matchArtistToDB

print("Full: {0}".format(artistDBToUseData.shape[0]))
idxs = artistDBToUseData["NumAlbums"] > 0
artistsToMatchData = artistDBToUseData.loc[idxs]
print(" Use: {0}".format(artistsToMatchData.shape[0]))

mrp = matchRunParams()
mmg = masterMatchGate(mdbDataDBToUse, dbToUse)
mmg.setArtistToMatchData(artistsToMatchData)
matchData = masterDBData(dtype="User", dbs="Master")
matchData.setUserDBData("Master", masterArtistAlbumsData["ArtistName"], masterArtistAlbumsData["NumAlbums"], masterArtistAlbumsData["Albums"])

In [ ]:
matchData.dbs

In [ ]:
tsMatch = timestat("Matching [{0}] Artists From [{1}]".format(artistsToMatchData.shape[0], dbToUse))

In [ ]:
from matchUtils import getUnmatchedArtistsFromDB, getArtistsToMatch, serialRun, multiProcRun
usePool = False
numProcs=3

numRunData = mmg.setArtistRunData()
tsName = timestat("Matching Artist Names")
print("  Matching {0} Artists".format(numRunData))
params={"artistCutoff": 0.85, "by": "Artist", 'matchData': matchData}
runResults = multiProcRun(mmg, numProcs=numProcs, params=params) if usePool else serialRun(mmg=mmg, params=params)
mmg.setMatchedArtistData(runResults)
mmg.updateArtistDataArtistRunStatus()
tsName.stop()
 
tsMatch.update()

In [ ]:
## Load Albums For Matched Names

tsIDX = timestat("Loading Album Data For Matched Index")
matchData.loadAlbums(idxReq=mmg.matchedArtistIDXReq)
tsIDX.stop()
tsMatch.update()

In [ ]:
## Match Artist Albums

%autoreload
from matchArtistToDB import matchArtistToDB
tsAlbum = timestat("Matching Artist Albums")
for runNum,(runDefs, runThresholds) in enumerate(mrp.getRunParams()):
    #print("="*125)
    #print("Run #: {0}".format(runNum))
    runThresholds['albumCutoff'] = 0.7
    runThresholds['numAlbums']   = 1
    runThresholds['score']       = 0.0
    runThresholds["by"]          = "Album"
    runThresholds['detail']      = 0
    runThresholds['matchData']   = matchData
    runStatus = True
    while runStatus is True:
        numArtistsToMatchForRun = mmg.setAlbumRunData(**runDefs)
        if numArtistsToMatchForRun["ForRun"] <= 0:
            runStatus = False
            continue
            
        print("  Matching {0} Artists For Run Number [{1}]".format(numArtistsToMatchForRun, runNum))        
        runResults = multiProcRun(mmg=mmg, numProcs=numProcs, params=runThresholds) if usePool else serialRun(mmg=mmg, params=runThresholds)
        mmg.setMatchedArtistAlbumData(runResults)
        mmg.updateArtistDataAlbumRunStatus()
tsAlbum.stop()
print("")

In [ ]:
def getAlbumMatchResults(matchedArtistAlbumData):
    dbToUseResults = {}
    for dbToUseID,dbToUseAlbumMatchData in matchedArtistAlbumData.items():
        dbToUseResults[dbToUseID] = {}
        for db,dbAlbumMatchData in dbToUseAlbumMatchData.items():
            if dbAlbumMatchData is None:
                continue
            albumResult = DataFrame(dbAlbumMatchData).T
            dbIDMatch   = albumResult[(albumResult["Match"] >= 2.0) & (albumResult["Exact"] >= 1.0)].sort_values(by=["Match","Score"], ascending=False).head(1)
            if dbIDMatch.shape[0] > 0:
                dbToUseResults[dbToUseID][db] = dbIDMatch.index[0]
    dbToUseResults = Series(dbToUseResults).apply(Series)
    return dbToUseResults

In [ ]:
matchedArtistData      = {item["ArtistID"]: item["Results"] for item in mmg.matchedArtistData}
matchedArtistAlbumData = {item["ArtistID"]: item["Results"] for item in mmg.matchedArtistAlbumData}
dbToUseResults         = getAlbumMatchResults(matchedArtistAlbumData)


In [ ]:
io = fileIO()
artistSummaryDF     = io.get("music_artistSummaryDF.p")
myMusicArtistAlbums = DataFrame({artistName: {"ArtistName": artistName, "Albums": Series(df["AlbumName"].to_list())} for artistName,df in artistSummaryDF.groupby("ArtistName")}).T

In [ ]:
myMusicMasterIDs = {}

N = myMusicArtistAlbums.shape[0]
ts = timestat("Matching {0} Artists From My Music".format(N))
matmdb = matchArtistToMasterDB(masterArtistAlbumsData)
for n,(idx,row) in enumerate(myMusicArtistAlbums.iterrows()):
    matmdb.setArtistInfo(row)
    matmdb.findArtistMatches()
    matmdb.findArtistAlbumMatches()
    
    myMusicMasterIDs[idx] = matmdb.getAlbumMatchResults()
    if len(myMusicMasterIDs) > 100:
        ts.update(n=n+1, N=N)
        break
ts.stop()

In [ ]:
matchedArtistData      = {item["ArtistID"]: item["Results"] for item in }
#matchedArtistAlbumData = {item["ArtistID"]: item["Results"] for item in mmg.matchedArtistAlbumData}
#dbToUseResults         = getAlbumMatchResults(matchedArtistAlbumData)


In [ ]:
def getAlbumMatchResults(matchedArtistAlbumData):
    dbToUseResults = {}
    for dbToUseID,dbToUseAlbumMatchData in matchedArtistAlbumData.items():
        dbToUseResults[dbToUseID] = {}
        for db,dbAlbumMatchData in dbToUseAlbumMatchData.items():
            if dbAlbumMatchData is None:
                continue
            albumResult = DataFrame(dbAlbumMatchData).T
            dbIDMatch   = albumResult[(albumResult["Match"] >= 2.0) & (albumResult["Exact"] >= 1.0)].sort_values(by=["Match","Score"], ascending=False).head(1)
            if dbIDMatch.shape[0] > 0:
                dbToUseResults[dbToUseID][db] = dbIDMatch.index[0]
    dbToUseResults = Series(dbToUseResults).apply(Series)
    return dbToUseResults

In [ ]:
def getBestMatch(matchResult):
    value  = matchResult[matchResult["Match"] >= 2.0]
    retval = None if value.shape[0] == 0 else list(value.head(1).index)[0]
    return retval

Series(myMusicMasterIDs).apply(getBestMatch)

In [ ]:
from matchArtistToDB import masterMatch

class matchArtistToMasterDB:
    def __init__(self, masterArtistAlbumsData, debug=False, detail=0):
        self.mdbData = masterArtistAlbumsData
        self.mdbDataSearchArtistNames = masterArtistAlbumsData["ArtistName"].str.upper()
        
        self.debug  = debug
        self.detail = detail
        
        self.matchNumArtistName     = None
        self.matchArtistNameCutoff  = None
        self.matchArtistAlbumCutoff = None
        self.matchNumArtistAlbums   = None
        self.matchScore             = None

        self.artistData             = None
        self.artistSearchResults    = {}
        self.albumSearchResults     = {}
        
        self.setThresholds()
        
        self.mm = masterMatch()
        
        
    #############################################################################################################################
    # Search Inputs
    #############################################################################################################################
    def setArtistInfo(self, artistData):
        self.artistData      = artistData
        self.artistName      = artistData["ArtistName"]
        self.artistID        = artistData.name
        self.artistAlbums    = artistData["Albums"][artistData["Albums"].notna()] if artistData.get("Albums") is not None else []
        self.artistNumAlbums = len(self.artistAlbums)
        
        
    #############################################################################################################################
    # I/O
    #############################################################################################################################
    def getArtistMatchResults(self):
        matchResults = {}
        if isinstance(self.artistSearchResults, DataFrame) and self.artistSearchResults.shape[0] > 0:
            dbMatchResult = self.artistSearchResults.sort_values(by="Levenshtein", ascending=False)
            matchResults  = dbMatchResult.T.to_dict()
        return {"ArtistID": self.artistID, "ArtistName": self.artistName, "Results": matchResults}
    
    
    def getAlbumMatchResults(self):
        matchResults = {}
        if isinstance(self.albumSearchResults, DataFrame) and self.albumSearchResults.shape[0] > 0:
            dbMatchResult = self.albumSearchResults.sort_values(by=["Match", "Score"], ascending=False)
            dbMatchResult["ArtistName"] = dbMatchResult.apply(lambda row: self.mdbData.loc[row.name, "ArtistName"], axis=1)
            matchResults = dbMatchResult.T.to_dict()
        return {"ArtistID": self.artistID, "ArtistName": self.artistName, "ArtistNumAlbums": self.artistNumAlbums, "Results": matchResults}
            
        
    def setThresholds(self, **kwargs):
        self.matchNumArtistName     = kwargs.get("numArtists", 2)
        self.matchArtistNameCutoff  = kwargs.get("artistCutoff", 0.8)
        self.matchNumArtistAlbums   = kwargs.get("numAlbums", 2)
        self.matchArtistAlbumCutoff = kwargs.get("albumCutoff", 0.9)
        self.matchScore             = kwargs.get("score", 1.8)
        
        
    #############################################################################################################################
    # Search Functions For Artists
    #############################################################################################################################
    def findArtistMatches(self):
        #searchArtists = self.mdbData.getDBSearchArtistNames(db)
        searchArtists = self.mdbDataSearchArtistNames
        
        ####### Only Look At Names Within Range Of Artist
        tmp = searchArtists.apply(len)
        artistNameUpper = self.artistName.upper()
        lenArtistName   = len(artistNameUpper)
        
        idxs = abs(tmp - lenArtistName) < 5
        searchArtists = searchArtists.loc[idxs]
        #searchResults = self.mm.matchNames(tomatch=searchResults, value=artistNameUpper)
        searchResults = searchArtists.apply(self.mm.getLevenshtein, x2=artistNameUpper)
        searchResults.name = "Levenshtein"
        idxs   = searchResults >= self.matchArtistNameCutoff
        result = DataFrame(searchArtists[idxs]).join(searchResults[idxs])
        self.artistSearchResults = result
        

    #############################################################################################################################
    # Search Functions For Artists + Albums
    #############################################################################################################################
    def findArtistAlbumMatches(self):
        ### 2nd: Score Each Match Using Albums
        idxResults = {}
        for idx,artistMatchData in self.artistSearchResults.iterrows():
            artistMatchAlbums = [str(x).upper() for x in self.mdbData.loc[idx, "Albums"] if x is not None]
            numAlbums         = len(artistMatchAlbums)
            idxResults[idx]   = {"NumAlbums": numAlbums}
            idxResults[idx]["Match"]  = -1.0
            idxResults[idx]["Exact"]  = -1.0
            idxResults[idx]["Score"]  = -1.0
            idxResults[idx]["M"]      = 0
            idxResults[idx]["Albums"] = artistMatchAlbums if self.detail >= 2 else 0
            if numAlbums > 0:
                try:
                    M    = self.artistAlbums.apply(lambda artistAlbum: Series([self.mm.getLevenshtein(matchAlbum, artistAlbum.upper()) for matchAlbum in artistMatchAlbums]))
                except:
                    print("ERROR Producing M matrix")
                    print("  MatchAlbums:",artistMatchAlbums)
                    print(" ArtistAlbums:",self.artistAlbums)
                    continue
                    
                try:
                    Max0 = M.max(axis=0)
                    Max1 = M.max(axis=1)
                except:
                    print("ERROR Producing M.max() matrix")
                    print("  MatchAlbums:",artistMatchAlbums)
                    print(" ArtistAlbums:",self.artistAlbums)
                    idxResults[idx]["M"]      = M if self.detail >= 1 else 0
                    continue

                try:
                    idxResults[idx]["Match"]  = min([(Max0 >= 0.825).sum(), (Max1 >= 0.825).sum()])
                    idxResults[idx]["Exact"]  = min([(Max0 >= 0.875).sum(), (Max1 >= 0.875).sum()])
                    idxResults[idx]["Score"]  = min([Max0[Max0 > self.matchArtistAlbumCutoff].sum(), Max1[Max1 > self.matchArtistAlbumCutoff].sum()])
                    idxResults[idx]["M"]      = M if self.detail >= 1 else 0
                    idxResults[idx]["Albums"] = artistMatchAlbums if self.detail >= 2 else 0
                except:
                    print("ERROR Producing Match/Exact/Score")
                    print("  MatchAlbums:",artistMatchAlbums)
                    print(" ArtistAlbums:",self.artistAlbums)
                    idxResults[idx]["M"]      = M if self.detail >= 1 else 0
                    continue
            idxResults[idx] = Series(idxResults[idx]).fillna(0)
        result = Series(idxResults).apply(Series) if len(idxResults) > 0 else DataFrame()
        #ts.stop()
        self.albumSearchResults = result

In [ ]:
len(myMusicMatchIDs)

In [ ]:
len(noMusicMatchs)

In [ ]:
multiMusicMatchs['AFI']

# Music Details

In [ ]:
from os import walk
import subprocess

def getPathSize(path,retUnit="M"):
    units = {"B": pow(1024,0), "K": pow(1024,1), "M": pow(1024,2), "G": pow(1024,3)}

    try:
        result = subprocess.check_output(['du','-sh', path]).split()[0].decode('utf-8')
    except:
        return None
    size   = float(result[:-1])
    unit   = result[-1]
    size  *= (units[unit])/(units[retUnit])
    return size


def getPathFiles(path):
    pathFiles = []
    pathDirs  = {}
    for i,(root, dirs, files) in enumerate(walk(path)):
        pathFiles += [dirUtil(root).join(ifile) for ifile in files]
        if len(dirs) > 0:
            pathDirs.update({root: dirs})
    return {"Files": pathFiles, "Dirs": pathDirs}


def getPathTimestamp(path):
    return fsPath(path).modified


def getPathSummary(albumPathData):
    nSubdirs = len(albumPathData['Dirs'])
    nFiles   = len(albumPathData["Files"])
    ftypes   = Series([fsInfo(ifile).path.suffix for ifile in albumPathData["Files"]]).value_counts().to_dict()
    return {"nSubdirs": nSubdirs, "nFiles": nFiles, "ftypes": ftypes}

In [ ]:
def getArtistSummaryData(artistAlbums, albumsSummaryData):
    artistSummaryData = {}
    for artistName,artistData in artistAlbums.items():
        artistSummaryData[artistName] = {}
        for albumType,albumTypeData in artistData.items():
            artistSummaryData[artistName][albumType] = {album: albumsSummaryData.loc[album] for album in albumTypeData}

    retval = {}
    for artistName,artistData in artistSummaryData.items():
        for albumType,albumTypeData in artistData.items():
            for album,albumData in albumTypeData.items():
                albumName = dirUtil(album).name.split(" :: ")[0]
                key   = (artistName,albumType,albumName)
                value = albumData
                if retval.get(key) is not None:
                    print(key)
                retval[key] = value

    df = DataFrame(retval).T
    df = df.reset_index().rename({'level_0': "ArtistName", 'level_1': "AlbumType", 'level_2': "AlbumName"}, axis=1)
    return df


def getSummaryDataRollup(x):
    colname = x.name
    if colname in ["Size", "nFiles", "nSubdirs"]:
        return {colname: x.astype(int).sum()}
    elif colname in ["Timestamp"]:
        return {colname: x.max()}
    elif colname in ["AlbumName", "AlbumType", "ArtistName"]:
        nunique = x.nunique()
        if nunique == 1:
            return {colname: x.unique()[0]}
        else:
            return {"n{0}s".format(colname[:-4]): nunique}
    elif colname in ["ftypes"]:
        return {colname: x.apply(Series).sum().to_dict()}
    else:
        raise ValueError("Not sure what to do with column [{0}]".format(colname))
    return 0


def getSummaryRollupDataFrame(df, byArtist=True):
    retval = []
    keys   = ["ArtistName","AlbumType","AlbumName"]
    gby    = keys[:1] if byArtist else keys[:2]
    ignore = keys[-2:] if byArtist else keys[-1:]
    for idx,idxDF in df.groupby(gby):
        result = {}
        for item in idxDF.apply(summary).values:
            if sum([x in item.keys() for x in ignore]) == 0:
                result.update(item)    
        retval.append(result)
        
    summaryDF = DataFrame(retval).fillna(0)
    summaryDF['nAlbums'] = summaryDF['nAlbums'].astype(int)
    return summaryDF

In [ ]:
from pandas import DataFrame, Series
from listUtils import getFlatList

tsAll = timestat("Collecting Artist + Album Details Information")

ts = timestat("Flattening Albums")
artistAlbums = {artistName: {albumType: albumTypeData.albums for albumType,albumTypeData in artistData.albumData.items()} for artistName,artistData in mmpdData.items()}
albumsList   = getFlatList([y for y in getFlatList([x.values() for x in artistAlbums.values()]) if len(y) > 0])
ts.stop()

ts = timestat("Getting Size For {0} Albums".format(len(albumsList)))
albumSizes = {album: getPathSize(album) for album in albumsList}
ts.stop()

ts = timestat("Getting Files/Subdirs For {0} Albums".format(len(albumsList)))
albumFiles = {album: getPathFiles(album) for album in albumsList}
ts.stop()

ts = timestat("Getting Timestamps For {0} Albums".format(len(albumsList)))
albumTimestamps = {album: getPathTimestamp(album) for album in albumsList}
ts.stop()

ts = timestat("Getting Summary For {0} Albums".format(len(albumsList)))
albumSummaries = {album: getPathSummary(albumData) for album,albumData in albumFiles.items()}
ts.stop()

ts = timestat("Joining Data For {0} Albums".format(len(albumsList)))
albumsSummaryData = DataFrame(albumSummaries).T
albumsSummaryData = albumsSummaryData.join(Series(albumTimestamps, name="Timestamp"))
albumsSummaryData = albumsSummaryData.join(Series(albumSizes, name="Size"))
ts.stop()

ts = timestat("Creating Summary DataFrame")
artistSummaryDF = getArtistSummaryData(artistAlbums, albumsSummaryData)
ts.stop()

tsAll.stop()

In [ ]:
io = fileIO()
io.save(idata=artistSummaryDF, ifile="music_artistSummaryDF.p")
io.save(idata=albumsSummaryData, ifile="music_albumsSummaryData.p")
io.save(idata=albumSummaries, ifile="music_albumSummaries.p")

In [ ]:
artistRollupDF = getSummaryRollupDataFrame(artistSummaryDF, byArtist=True)

In [ ]:
"""
('A Flock Of Seagulls', 'Match', 'Greatest Hits Remixed')
('A Flock Of Seagulls', 'Match', 'I Ran')
('A Flock Of Seagulls', 'Match', 'Magic')
('A Global Threat', 'Match', 'Here We Are')
('A Tribe Called Quest', 'Match', 'Midnight Marauders')
('A Tribe Called Quest', 'Match', 'The Love Movement')
('A Tribe Called Quest', 'Match', 'The Low End Theory')
('A Tribe Called Quest', 'Match', 'The Lost Tribes')
('ABC', 'Match', 'Alphabet City')
('ABC', 'Match', 'Skyscraping')
('AFI', 'Match', 'A Fire Inside EP')
('Abdullah', 'Match', 'Graveyard Poetry')
('Abdullah', 'Match', 'Abdullah')
('Abominator', 'Match', "Damnation's Prophecy")
('Above The Law', 'Match', 'Call It What U Want')
('Abscess', 'Match', 'Tormented')
('Abscess', 'Match', 'Urine Junkies')
('Absolution', 'Match', 'Complete Discography')
('Action League', 'Match', 'Interrupt This Program')
('Adam Ant', 'Match', 'Antics In The Forbidden Zone')
('Addrisi Brothers', 'Match', 'Cherrystone')
('Adele', 'Match', '25')
('Adele', 'Match', 'Chasing Pavements')
('Adele', 'Match', 'Cold Shoulder')
('Adele', 'Match', 'Make You Feel My Love')
('Adele', 'Match', 'Rolling In The Deep')
('Adele', 'Match', 'Rumour Has It')
('Adele', 'Match', 'Skyfall')
('Adele', 'Match', 'Someone Like You')
('Adele', 'Match', 'Turning Tables')
('Adele', 'Match', 'iTunes Festival: London 2011')
('Adele', 'Match', 'iTunes Live From SoHo')
('Adele', 'Match', 'Hometown Glory')
('Aerosmith', 'Match', "Aerosmith's Greatest Hits")
('Agent 99', 'Match', 'Little Pieces: 1993-1995')
('Agnostic Front', 'Match', 'Riot, Riot, Upstart')
('Air Supply', 'Match', 'Greatest Hits Live... Now And Forever')
('Air Supply', 'Match', 'The Definitive Collection')
('Akon', 'Match', 'Freedom')
('Al Green', 'Match', 'Anthology')
('Al Green', 'Match', 'Call Me')
('Alan Jackson', 'Match', 'Everything I Love')
('Alannah Myles', 'Match', 'A-Lan-Nah')
('Alannah Myles', 'Match', 'Arival')
('Alannah Myles', 'Match', 'Alannah Myles')
('Alannah Myles', 'Match', 'Black Velvet')
('Albert Hammond', 'Match', 'Albert Hammond')
('Alex Britti', 'Match', 'It.pop')
('Alice In Chains', 'Match', 'Facelift')
('Alice In Chains', 'Match', 'Live')
('Alice In Chains', 'Match', 'The Devil Put Dinosaurs Here')
('Alice In Chains', 'Match', 'Bank Heist')
('Alice In Chains', 'Match', 'Music Bank')
('Alice In Chains', 'Match', 'Nothing Safe: The Best Of The Box')
('Alice In Chains', 'Match', 'The Music Of Alice In Chains')
('Alice In Chains', 'Match', 'Sap')
('Alice In Chains', 'Match', 'Dirt')
('All Out War', 'Match', 'For Those Who Were Crucified')
('All-4-One', 'Match', 'I Can Love You Like That')
('Alley Boy', 'Match', 'Nigganati')
('Aloha', 'Match', 'Sugar')
('Aloha', 'Match', 'Aloha')
('Alphaville', 'Match', 'Salvation')
('Amanda Marshall', 'Match', 'Amanda Marshall')
('Amber Asylum', 'Match', 'Songs Of Sex And Death')
('Amber Asylum', 'Match', 'The Supernatural Parlour Collection')
('Amel Larrieux', 'Match', 'Infinite Possibilities')
('America', 'Match', 'Greatest Hits')
('Amoeba', 'Match', 'Pivot')
('Amoeba', 'Match', 'Watchful')
('Amorphis', 'Match', 'Am Universum')
('Amorphis', 'Match', 'Elegy')
('Amorphis', 'Match', 'The Karelian Isthmus')
('Amorphis', 'Match', 'Tuonela')
('Amorphis', 'Match', 'Black Winter Day')
('Amorphis', 'Match', 'My Kantele')
('Amorphis', 'Match', 'Privilege Of Evil')
('Amsvartner', 'Match', 'Dreams')
('Amy Grant', 'Match', 'In Concert')
('Amy Grant', 'Match', 'In Motion (The Remixes)')
('Amy Grant', 'Match', "My Father's Eyes")
('Amy Grant', 'Match', 'Unguarded')
('Amy Grant', 'Match', 'My Best Christmas')
('Amy Grant', 'Match', 'The Collection')
('Amy Grant', 'Match', 'Breath Of Heaven')
('Amy Grant', 'Match', 'Big Yellow Taxi')
('Amy Grant', 'Match', 'Lead Me On')
('Amy Grant', 'Match', 'Simple Things')
('Amy Grant', 'Match', 'Tennessee Christmas')
('Amy Rigby', 'Match', 'The Sugar Tree')
('Anacrusis', 'Match', 'Manic Impressions')
('Anal Cunt', 'Match', 'Everyone Should Be Killed')
('Anal Cunt', 'Match', 'I Like It When You Die')
('Anal Cunt', 'Match', 'It Just Gets Worse')
('Anal Cunt', 'Match', 'Top 40 Hits')
('Anasarca', 'Match', 'Moribund')
('Ancient', 'Match', 'Mad Grandiose Bloodfiends')
('Ancient', 'Match', 'Proxima Centauri')
('Ancient', 'Match', 'The Cainian Chronicle')
('Ancient', 'Match', 'The Halls Of Eternity')
('And Oceans', 'Match', 'The Symmetry Of I The Circle Of O')
('Andre Williams', 'Match', 'Silky')
('Andrea Marcovicci', 'Match', 'Marcovicci Sings Movies')
('Andy M. Stewart', 'Match', 'Fire In The Glen')
('Angelina Réaux', 'Match', 'Stranger Here Myself: Songs of Kurt Weill')
('Angizia', 'Match', 'Das Schachbrett Des Trommelbuben Zacharias')
('Angizia', 'Match', 'Die Kemenaten Scharlachroter Lichter')
('Ani DiFranco', 'Match', 'Little Plastic Castle')
('Anita Baker', 'Match', 'The Songstress')
('Anita Baker', 'Match', 'The Songstress')
('Anonymous 4', 'Match', "Love's Illusion")
('Anorexia Nervosa', 'Match', 'Exile')
('Anthony Moore', 'Match', "Flying Doesn't Help")
('Antichrisis', 'Match', 'Perfume')
('Anvil', 'Match', 'Absolutely No Alternative')
('Anvil', 'Match', 'Still Going Strong')
('Anvil', 'Match', 'This Is Thirteen')
('Anvil', 'Match', 'Backwaxed')
('Apoptygma Berzerk', 'Match', '7')
('Aqua', 'Match', 'Aquarius')
('Aqua', 'Match', 'Bubble Mix')
('Aqua', 'Match', 'Cartoon Heroes')
('Aqua', 'Match', 'Aquarium')
('Aqua', 'Match', 'Megalomania')
('Aqua', 'Match', 'Bumble Bees')
('Aqua', 'Match', 'Barbie Girl')
('Aqua', 'Match', 'My Oh My')
('Aqua', 'Match', 'Roses Are Red')
('Arctic Monkeys', 'Match', 'AM')
('Arctic Monkeys', 'Match', 'Favourite Worst Nightmare')
('Arctic Monkeys', 'Match', 'Humbug')
('Arctic Monkeys', 'Match', "Whatever People Say I Am, That's What I'm Not")
('Area 54', 'Match', 'No Visible Scars')
('Aretha Franklin', 'Match', 'Amazing Grace')
('Aretha Franklin', 'Match', 'I Knew You Were Waiting (For Me)')
('Aretha Franklin', 'Match', '30 Greatest Hits')
('Aretha Franklin', 'Match', "Aretha's Gold")
('Ariana Grande', 'Match', 'Yours Truly')
('Ariana Grande', 'Match', 'Santa Tell Me')
('Arkham 13', 'Match', 'The Wrath')
('Arma Angelus', 'Match', 'Where Sleeplessness Is Rest From Nightmares')
('Armored Saint', 'Match', 'Revelation')
('Armored Saint', 'Match', 'Nod To The Old School')
('Armored Saint', 'Match', 'Saints Will Conquer')
('Arthur Lyman', 'Match', 'Hawaiian Sunset')
('As All Die', 'Match', 'Time Of War And Conflict')
('As Friends Rust', 'Match', 'Won')
('As Friends Rust', 'Match', 'The Fists Of Time An Anthology Of Short Fiction And Non-Fiction')
('As Friends Rust', 'Match', 'As Friends Rust')
('Ashes', 'Match', 'And The Angels Wept')
('Ashes', 'Match', 'Death Has Made Its Call')
('Asia', 'Match', 'Aria')
('Asia', 'Match', 'Aura')
('Asia', 'Match', 'Aqua')
('Aspera Ad Astra', 'Match', 'Peace')
('Assemblage 23', 'Match', 'Contempt')
('Assemblage 23', 'Match', 'Failure')
('Assert', 'Match', 'Insurrection Rocks')
('Assück', 'Match', 'Misery Index')
('Astroqueen', 'Match', 'Into Submission')
('At The Drive-In', 'Match', 'Vaya')
('Athenaeum', 'Match', 'Radiance')
('Ativin', 'Match', 'Pills Versus Planes')
('Atlantic Starr', 'Match', 'Atlantic Starr')
('Atlantic Starr', 'Match', 'Brilliance')
('Atlantic Starr', 'Match', 'Legacy')
('Atlantic Starr', 'Match', 'Radiant')
('Atlantic Starr', 'Match', 'Time')
('Atlantic Starr', 'Match', "We're Movin' Up")
('Atlantic Starr', 'Match', 'Love Crazy')
('Atlantic Starr', 'Match', 'One Lover At A Time')
('Atlantic Starr', 'Match', 'Yours Forever')
('Atlantic Starr', 'Match', 'Ultimate Collection')
('Atlantis Ita', 'Match', 'The Good Land')
('Atomic Kitten', 'Match', 'Right Now')
('Atomic Opera', 'Match', 'Gospel Cola')
('Atrax Morgue', 'Match', 'Sickness Report')
('Atrox', 'Match', 'Contact')
('Atrox', 'Match', 'Terrestrials')
('Aurora', 'Match', 'Eos - Sadiam')
('Autograph', 'Match', 'Missing Pieces')
('Autonomy', 'Match', 'Autonomy')
('Avant', 'Match', 'My Thoughts')
('Avicii', 'Match', 'True')
('Avskum', 'Match', 'In The Spirit Of Mass Destruction')
('Ayahuasca', 'Match', 'Digital Alchemy')
('Annie Lennox', 'Match', 'A Christmas Cornucopia')
('Annie Lennox', 'Match', 'Medusa + Live In Central Park')
('Annie Lennox', 'Match', 'A Whiter Shade Of Pale')
('Annie Lennox', 'Match', 'Cold')
('Annie Lennox', 'Match', 'Little Bird')
('Annie Lennox', 'Match', 'The Annie Lennox Collection')
('Ace of Base', 'Match', 'Greatest Hits')
('Ace of Base', 'Match', 'Singles Of The 90s')
('Ace of Base', 'Match', 'The Collection')
('Ace of Base', 'Match', 'Happy Nation')
('Ace of Base', 'Match', 'The Golden Ratio')
('Ace of Base', 'Match', 'The Sign')
('Ace of Base', 'Match', 'Happy Nation')
('Armin Van Buuren', 'Match', '10 Years')
('Meat Loaf', 'Match', 'Bad Attitude')
('Meat Loaf', 'Match', 'Blind Before I Stop')
('Meat Loaf', 'Match', 'Dead Ringer')
('Meat Loaf', 'Match', 'Hang Cool Teddy Bear')
('Meat Loaf', 'Match', 'Hell in a Handbasket')
('Meat Loaf', 'Match', 'Midnight at the Lost and Found')
('B.J. Thomas', 'Match', 'Hooked On A Feeling: Greatest & Latest')
('Babylonian Tiles', 'Match', 'Teknicolour Aftermath')
('Backfire!', 'Match', 'Still Dedicated')
('Backstreet Boys', 'Match', "Backstreet's Back")
('Backstreet Boys', 'Match', 'Never Gone')
('Backstreet Boys', 'Match', 'Unbreakable')
('Backstreet Boys', 'Match', 'Selections From A Night Out With The Backstreet Boys')
('Bad Manners', 'Match', 'Fat Sound')
('Bad Manners', 'Match', 'Heavy Petting')
('Bad Times', 'Match', 'Bad Times')
('Bad Trip', 'Match', 'Fear And Loathing')
('Badfinger', 'Match', 'Head First')
('Bagman', 'Match', 'Wrap')
('Balaam And The Angel', 'Match', 'Prime Time')
('Banana Erectors', 'Match', 'Banana Erectors')
('Bane', 'Match', 'Give Blood')
('Bane', 'Match', 'Holding This Moment')
('Bang Tango', 'Match', 'Live')
('Barenaked Ladies', 'Match', 'Gordon')
('Barenaked Ladies', 'Match', 'Rock Spectacle')
('Barenaked Ladies', 'Match', 'Pinch Me')
('Barenaked Ladies', 'Match', 'Disc One: All Their Greatest Hits (1991-2001)')
('Barracudas', 'Match', 'Drop Out With The Barracudas')
('Barracudas', 'Match', 'Through The Mysts Of Time')
('Barry White', 'Match', 'Barry & Glodean')
('Barry White', 'Match', 'Barry White The Man')
('Barry White', 'Match', 'Beware!')
('Barry White', 'Match', 'Dedicated')
('Barry White', 'Match', 'Is This Whatcha Wont?')
('Barry White', 'Match', "Stone Gon'")
('Barry White', 'Match', 'The Icon Is Love')
('Barry White', 'Match', 'The Message Is Love')
('Barry White', 'Match', 'The Right Night & Barry White')
('Barry White', 'Match', 'Gold - The Very Best Of')
('Barry White', 'Match', 'Just For You')
('Barry White', 'Match', 'Love Songs')
('Barry White', 'Match', 'Change')
('Barry White', 'Match', 'Sheet Music')
('Barry White', 'Match', 'Staying Power')
('Barry White', 'Match', 'Under The Influence Of Love')
('Bassholes', 'Match', 'The Secret Strength Of Depression (Live At KSPC, Claremont)')
('Beastie Boys', 'Match', 'Hot Sauce Committee Part Two')
('Beastie Boys', 'Match', 'Ill Communication')
('Beastie Boys', 'Match', "Paul's Boutique")
('Beastie Boys', 'Match', 'The Mix-Up')
('Beaver Nelson', 'Match', 'Little Brother')
('Beck', 'Match', 'Golden Feelings')
('Beck', 'Match', 'Guerolito')
('Beck', 'Match', 'Modern Guilt')
('Beck', 'Match', 'Guero')
('Beck', 'Match', 'Morning Phase')
('Beenie Man', 'Match', 'Beenie Man Meets Mad Cobra')
('Beenie Man', 'Match', 'Cool Cool Rider')
('Beenie Man', 'Match', 'Dis Unu Fi Hear')
('Beenie Man', 'Match', 'Gold')
('Beenie Man', 'Match', 'Many Moods Of Moses')
('Beenie Man', 'Match', 'Tropical Storm')
('Beenie Man', 'Match', 'Undisputed')
('Beenie Man', 'Match', 'Unstoppable')
('Beenie Man', 'Match', "Ruff 'N' Tuff")
('Beenie Man', 'Match', 'The Magnificent')
('Beenie Man', 'Match', 'Y2K')
('Beenie Man', 'Match', 'Youth Quake')
('Beenie Man', 'Match', 'Blessed')
('Beenie Man', 'Match', 'Defend It')
('Beenie Man', 'Match', 'Maestro')
('Beenie Man', 'Match', 'The Doctor')
('Beezus', 'Match', 'Breakfast Was Weird')
('Beherit', 'Match', 'Messe Des Morts')
('Bei Maejor', 'Match', 'Upside Down Vol 2')
('Belphegor', 'Match', 'Necrodaemon Terrorsathan')
('Ben Folds', 'Match', 'Ben Folds Live')
('Ben Folds', 'Match', 'Supersunnyspeedgraphic, The LP')
('Ben Folds', 'Match', 'Speed Graphic')
('Ben Folds', 'Match', 'Sunny 16')
('Ben Folds', 'Match', 'Super D')
('Ben Harper', 'Match', 'Get Up!')
('Ben Harper', 'Match', "Give Till It's Gone")
('Ben Watt', 'Match', 'North Marine Drive + Summer Into Winter')
('Benjamin B.', 'Match', 'Forever Careless')
('Benny Joy', 'Match', '"Rockin\' And Rollin\' With Benny Joy')
('Bent Leg Fatima', 'Match', 'Bent Leg Fatima')
('Berlin', 'Match', 'Best Of Berlin 1979 - 1988')
('Bert Jansch', 'Match', 'Toy Balloon')
('Beseech', 'Match', 'Black Emotions')
('Beth Capper', 'Match', 'Complimentary Mood Enhancer (Presents The Original Soundtrack For The Motion Picture "Wig-Lickers" Based On The True Story Of Trigger)')
('Beth Hart', 'Match', 'Live At The Royal Albert Hall')
('Beth Hart', 'Match', 'Bang Bang Boom Boom')
('Beth Hart', 'Match', '37 Days')
('Bethzaida', 'Match', 'LXXVIII')
('Bethzaida', 'Match', 'Nine Worlds')
('Bette Midler', 'Match', 'Beaches (Original Soundtrack Recording)')
('Better Than Ezra', 'Match', 'All Together Now')
('Better Than Ezra', 'Match', 'Closer')
('Better Than Ezra', 'Match', 'Paper Empire')
('Better Than Ezra', 'Match', 'Artifakt')
('Better Than Ezra', 'Match', 'Greatest Hits')
('Beulah', 'Match', 'When Your Heartstrings Break')
('Beyond Dawn', 'Match', 'Pity Love')
('Beyond', 'Match', 'Reassemble')
('Big Ass Truck', 'Match', 'The Rug')
('Big Brother & The Holding Company', 'Match', "Can't Go Home Again")
('Big Brother & The Holding Company', 'Match', 'Do What You Love')
('Big Electric Cat', 'Match', 'Eyelash')
('Big K.R.I.T.', 'Match', "It's Better This Way")
('Big L', 'Match', 'Deadly Combination')
('Blackstreet', 'Match', 'Another Level')
('Boyz II Men', 'Match', 'Evolution')
('Big Head Todd', 'Match', 'Sister Sweetly')
('Carpenters', 'Match', 'Now & Then')
('Count Basie', 'Match', 'Basie Jam #3')
('Dave Brubeck', 'Match', 'Time In')
('DJ Shadow', 'Match', "This Time (I'm Gonna Try It My Way)")
('DJ Shadow', 'Match', "Mashin' On The Motorway")
('Dr.Alban', 'Match', "It's My Life")
('David Cassidy', 'Match', 'Then and Now')
('Edguy', 'Match', 'Theater of Salvation')
('Eisbrecher', 'Match', 'Die Hölle Muss Warten')
('Fatboy Slim', 'Match', 'The Rockafeller Skank')
('Galactic Cowboys', 'Match', 'Let It Go')
('Galactic Cowboys', 'Match', 'Machine Fish')
('Galactic Cowboys', 'Match', 'The Horse That Bud Bought')
('Glen Hansard', 'Match', 'Rhythm and Repose')
('Ginuwine', 'Match', 'Greatest Hits')
('Herbie Hancock', 'Match', 'Maiden Voyage')
('INXS', 'Match', 'Never Tear Us Apart')
('Jaci Velasquez', 'Match', 'Jaci Velasquez')
('Jimmy Eat World', 'Match', 'Jimmy Eat World')
('Kanye West', 'Match', 'Stronger')
('Kim Wilde', 'Match', 'The Singles Collection 1981-1993')
('Kim Wilde', 'Match', 'The Very Best of Kim Wilde')
('Lenny Kravitz', 'Match', 'Greatest Hits')
('Luke Combs', 'Match', 'What You See Is What You Get')
('Lil Uzi Vert', 'Match', 'Luv Is Rage 2')
('Maxx', 'Match', 'Get-A-Way')
('Michael W. Smith', 'Match', 'Christmas')
('Michael W. Smith', 'Match', 'Worship')
('Muse', 'Match', 'HAARP')
('Muse', 'Match', 'Butterflies & Hurricanes')
('Muse', 'Match', 'Sing For Absolution')
('Muse', 'Match', 'Hullabaloo Soundtrack')
('Nazareth', 'Match', 'Nazareth')
('Nazareth', 'Match', 'Greatest Hits')
('Nickelback', 'Match', "Savin' Me")
('Notorious B.I.G.', 'Match', 'Born Again')
('Notorious B.I.G.', 'Match', 'Duets: The Final Chapter')
('New Found Glory', 'Match', 'Makes Me Sick')
('Pantera', 'Match', 'Planet Caravan')
('Paul McCartney', 'Match', 'Run Devil Run')
('Pink Floyd', 'Match', 'Relics')
('Reba McEntire', 'Match', 'If You See Him')
('Reba McEntire', 'Match', 'Reba McEntire')
('Reba McEntire', 'Match', 'So Good Together')
('Reba McEntire', 'Match', 'The Last One to Know')
('Reba McEntire', 'Match', 'Unlimited')
('Red Hot Chili Peppers', 'Match', 'Californication')
('Rick Astley', 'Match', 'Whenever You Need Somebody')
('Robbie Williams', 'Match', 'Rudebox')
('Ryan Adams', 'Match', '29')
('Ryan Adams', 'Match', 'Demolition')
('Ryan Adams', 'Match', 'Easy Tiger')
('Ryan Adams', 'Match', 'Gold')
('Ryan Adams', 'Match', 'Heartbreaker')
('Seal', 'Match', 'Seal')
('Slayer', 'Match', 'Show No Mercy')
('Sophie B. Hawkins', 'Match', 'Whaler')
('Staind', 'Match', '14 Shades of Grey')
('Staind', 'Match', 'Break the Cycle')
('Sum 41', 'Match', 'Underclass Hero')
('Tracy Chapman', 'Match', 'New Beginning')
('Tim McGraw', 'Match', 'Number One Hits')
('Trace Adkins', 'Match', "Something's Going On")
('The Band Perry', 'Match', 'Pioneer')
('The Manhattan Transfer', 'Match', 'Bop Doo-Wopp')
('Vince Guaraldi Trio', 'Match', 'A Charlie Brown Christmas')
('Vampire Weekend', 'Match', 'Holiday')
('Vengaboys', 'Match', 'Boom, Boom, Boom, Boom!!')
('W.A.S.P.', 'Match', 'Helldorado')
('W.A.S.P.', 'Match', 'Kill Fuck Die')
('Wayne Shorter', 'Match', 'Atlantis')
('Wayne Shorter', 'Match', 'Native Dancer')
('Wayne Shorter', 'Match', 'Phantom Navigator')
"""